In [1]:
# Importing necessary modules
from tensorflow import keras
from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from keras.regularizers import l2
import matplotlib.pyplot as plt
from plot_keras_history import show_history
import mlflow
import mlflow.keras

2024-05-18 23:31:45.401745: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 23:31:45.510308: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 23:31:45.945755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 23:31:47.116867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
num_classes = 10

x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape, 'train input samples')
print(x_test.shape, 'test input samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape, 'train output samples')
print(y_test.shape, 'test output samples')


(50000, 32, 32, 3) train input samples
(10000, 32, 32, 3) test input samples
(50000, 10) train output samples
(10000, 10) test output samples


In [3]:
# Function to run 10 sub experiments
def run_sub_experiments(x_train,x_test,y_train,y_test):
    with mlflow.start_run(run_name="CIFAR10") as parent_run: 
        
    # Simple CNN with Only One Convolutional Layer
        with mlflow.start_run(run_name="Sub-Experiment-1", nested=True):
            mlflow.keras.autolog()
            
            model = keras.Sequential()
            model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
            model.add(MaxPooling2D((2, 2)))
            model.add(Flatten())
            model.add(Dense(128, activation='relu'))
            model.add(Dense(10, activation='softmax'))
            model.summary()
            
            model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
            
            show_history(history)
            
            loss, acc = model.evaluate(x_test, y_test, verbose=2)
            print("Test accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Test_Accuracy": acc})
            loss, acc = model.evaluate(x_train, y_train, verbose=2)
            print("Train accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Train_Accuracy": acc})

            model.save('models/model_1.keras')

        # CNN with Only three Convolutional Layers
        with mlflow.start_run(run_name="Sub-Experiment-2", nested=True):
            mlflow.keras.autolog()
            
            model1 = keras.Sequential()
            model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
            model1.add(MaxPooling2D((2, 2)))
            model1.add(Conv2D(64, (3, 3), activation='relu'))
            model1.add(MaxPooling2D((2, 2)))
            model1.add(Conv2D(64, (3, 3), activation='relu'))
            model1.add(Flatten())
            model1.add(Dense(64, activation='relu'))
            model1.add(Dense(10, activation='softmax'))
            model1.summary()
            
            model1.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history1 = model1.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
            
            show_history(history1)
            
            loss, acc = model1.evaluate(x_test, y_test, verbose=2)
            print("Test accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Test_Accuracy": acc})
            loss, acc = model1.evaluate(x_train, y_train, verbose=2)
            print("Train accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Train_Accuracy": acc})

            model1.save('models/model_2.keras')
    
        # CNN with dropout
        with mlflow.start_run(run_name="Sub-Experiment-3", nested=True):
            mlflow.keras.autolog()
        
            model2 = keras.Sequential()
            model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
            model2.add(MaxPooling2D((2, 2)))
            model2.add(Dropout(0.25))
            model2.add(Conv2D(64, (3, 3), activation='relu'))
            model2.add(MaxPooling2D((2, 2)))
            model2.add(Dropout(0.25))
            model2.add(Conv2D(64, (3, 3), activation='relu'))
            model2.add(Flatten())
            model2.add(Dense(64, activation='relu'))
            model2.add(Dropout(0.5))
            model2.add(Dense(10, activation='softmax'))
            model2.summary()
            
            model2.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history2 = model2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
            
            show_history(history2)
            
            loss, acc = model2.evaluate(x_test, y_test, verbose=2)
            print("Test accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Test_Accuracy": acc})
            loss, acc = model2.evaluate(x_train, y_train, verbose=2)
            print("Train accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Train_Accuracy": acc})

            model2.save('models/model_3.keras')
        
        # Using learning rates
        with mlflow.start_run(run_name="Sub-Experiment-4", nested=True):
            mlflow.keras.autolog()
            
            model3 = keras.Sequential()
            model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
            model3.add(MaxPooling2D((2, 2)))
            model3.add(Conv2D(64, (3, 3), activation='relu'))
            model3.add(MaxPooling2D((2, 2)))
            model3.add(Flatten())
            model3.add(Dense(256, activation='relu'))
            model3.add(Dense(10, activation='softmax'))
            model3.summary()
            
            opt_new = keras.optimizers.SGD(learning_rate=0.1)
            model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
            history3 = model3.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

            show_history(history3)
            
            loss, acc = model3.evaluate(x_test, y_test, verbose=2)
            print("Test accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Test_Accuracy": acc})
            loss, acc = model3.evaluate(x_train, y_train, verbose=2)
            print("Train accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Train_Accuracy": acc})

            model3.save('models/model_4.keras')
            
        
        # Using l2 regularization
        with mlflow.start_run(run_name="Sub-Experiment-5", nested=True):
            mlflow.keras.autolog()
        
            model4 = keras.Sequential()
            model4.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(32, 32, 3)))
            model4.add(MaxPooling2D((2, 2)))
            model4.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
            model4.add(MaxPooling2D((2, 2)))
            model4.add(Flatten())
            model4.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
            model4.add(Dense(10, activation='softmax'))
            model4.summary()
            
            model4.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history4 = model4.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
            
            show_history(history4)
            
            loss, acc = model4.evaluate(x_test, y_test, verbose=2)
            print("Test accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Test_Accuracy": acc})
            loss, acc = model4.evaluate(x_train, y_train, verbose=2)
            print("Train accuracy: {:5.2f}%".format(100*acc))
            mlflow.log_metrics({"Train_Accuracy": acc})

            model4.save('models/model_5.keras')
        
        
       
        
        
        



In [ ]:
run_sub_experiments(x_train,x_test,y_train,y_test)    